In [ ]:
#importing libraries
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

matches = pd.read_csv("C:\\Users\\moham\\Downloads\\data.csv")
deliveries = pd.read_csv("C:\\Users\\moham\\Downloads\\deliveries.csv")

In [ ]:
matches.sample(10)

In [ ]:
deliveries.sample(10)

In [ ]:
total_runs_df = deliveries.groupby(['match_id','inning'])['total_runs'].sum().reset_index()
total_runs_df = total_runs_df[total_runs_df['inning']==1]
total_runs_df.sample(10)


In [ ]:
merged_df = total_runs_df.merge(matches,left_on='match_id',right_on='id').drop(columns=['player_of_match','umpire1','umpire2','umpire3','toss_winner','toss_decision','date','season','venue','id'])
merged_df = merged_df[merged_df['dl_applied']==0]
merged_df = merged_df[merged_df['result']=='normal']
merged_df.drop(columns=['result','dl_applied','inning'],inplace=True)
print(merged_df.shape)
merged_df.sample(5)

In [ ]:
merged_df2 = merged_df.merge(deliveries,left_on='match_id',right_on='match_id')
merged_df2['Current_Runs'] = merged_df2.groupby(['match_id','inning'])['total_runs_y'].cumsum()
print(merged_df2.shape)
merged_df2.head(5)

In [ ]:
import warnings
warnings.filterwarnings(message='A value is trying to be set on a copy of a slice from a DataFrame.',action='ignore')
pd.options.mode.chained_assignment = None 
merged_inning2 = merged_df2[merged_df2['inning']==2]
merged_inning2['Required_Runs'] = merged_inning2['total_runs_x'] - merged_inning2['Current_Runs']
merged_inning2['Required_Runs'] = merged_inning2['Required_Runs']+1
print(merged_inning2.shape)
merged_inning2.head(10)

In [ ]:

pd.options.mode.chained_assignment = None 
merged_inning2['player_dismissed']=merged_inning2['player_dismissed'].fillna(0)
merged_inning2['player_dismissed']=merged_inning2['player_dismissed'].apply(lambda x:0 if x==0 else 1)
merged_inning2.head(5)

In [ ]:
pd.options.mode.chained_assignment = None 
merged_inning2['current_balls_bowled'] = ((merged_inning2['over']-1)*6)+merged_inning2['ball']
merged_inning2['balls_left'] = 120 - merged_inning2['current_balls_bowled']
merged_inning2['current_run_rate'] = merged_inning2['Current_Runs']/((merged_inning2['current_balls_bowled']/6))
merged_inning2['required_run_rate'] = merged_inning2['Required_Runs']/((merged_inning2['balls_left']/6))
merged_inning2.drop(columns=['dismissal_kind','fielder','win_by_runs',	'win_by_wickets',	'team1','team2','batsman','non_striker','bowler','is_super_over','wide_runs','bye_runs','legbye_runs','noball_runs','penalty_runs','batsman_runs','extra_runs'],axis=1,inplace=True)
merged_inning2.head(10)

In [ ]:
merged_inning2['win'] = merged_inning2.apply(
    lambda row: 1 if row['winner'] == row['batting_team'] else 0, axis=1
)
print(merged_inning2.shape)
merged_inning2.sample(5)

In [ ]:
pd.options.mode.chained_assignment = None 
temp = merged_inning2.groupby('match_id')['player_dismissed'].cumsum().reset_index()

In [ ]:
temp.head(20)

In [ ]:
merged_inning2['index'] = [x for x in range(0,merged_inning2.shape[0])]
merged_inning2.head(20)

In [ ]:
merged_inning2.set_index('index',inplace=True)
merged_inning2.drop(columns=['player_dismissed'],axis=1,inplace=True)
merged_inning2.head(20)

In [ ]:
data = pd.concat([temp,merged_inning2],axis=1)
final_df = data
final_df.rename(columns={'player_dismissed':'wickets_down','total_runs_x':'target_score'},inplace=True)
columns = final_df.columns
print(columns)

In [ ]:
final_df.head(20)

In [ ]:
final_df.columns

In [ ]:
final_df.drop(columns=['index','match_id','winner','inning','total_runs_y'],axis=1,inplace=True)
final_df.head(10)

In [ ]:
final_df.shape

Now before moving any further, let's check if our data has some missing values or some infinite values, it is necessary to remove them as it will cause difficulties in future model training

In [ ]:
final_df.isnull().sum()

In [ ]:
final_df.dropna(inplace=True)
final_df.describe()

In [ ]:
final_df= final_df[final_df['balls_left']!=0]
final_df.describe()

In [ ]:
print(final_df.shape)
final_df.isnull().sum()

In [ ]:
final_df.sample()

Now preprocessing is completely done, and we are good to go. Data has been completely reduced to the form it should be in. Now data has all required columns which are necessary for the model training. Now we'll apply ml processings and functions on it to have a good model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(final_df.drop(columns=['win'],axis=1),final_df['win'],random_state=0,test_size=0.2)
print(f"X_train shape : {X_train.shape}\nX_test shape : {X_test.shape}\ny_train shape : {y_train.shape}\ny_test shape : {y_test.shape}")

In [ ]:
trf = ColumnTransformer([('OneHotEncoding',OneHotEncoder(drop='first',sparse_output=False),['city','batting_team','bowling_team'])],remainder='passthrough')

pipe = Pipeline(steps=[('encoding',trf),('model_training',LogisticRegression(solver='liblinear'))])

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
ypred = pipe.predict(X_test)
print('accuracy score is : ',int(accuracy_score(ypred,y_test)*100),'%')

Now all we needed to do with the data has been done. We have cleaned the data and also trained an ml model, it's time to convert it into a web app. For that I'll use pickle to dump my ml model object in a new file and we'll work there. ^_^

In [ ]:
import pickle
pickle.dump(pipe,open('ipl.pkl','wb'))

In [ ]:
final_df['bowling_team'].unique()

In [ ]:
final_df['batting_team'].unique()

In [ ]:
final_df['city'].unique()